# The Mathematical Engineering of Deep Learning

## Practical 1 (Python version)
**For an R or Julia version see the [course website](https://deeplearningmath.org/)**.

---

In this practical we will carry out some basic EDA and analysis of some popular ML Datasets that will be used in the course.

---

Make sure that the following packages are already installed on your system (before the practical starts):

* keras
* pytorch
* matplotlib
* numpy

### <font color='blue'>**Fashion MNIST**</font>

#### **Load using keras**

In [ ]:
from keras.datasets import fashion_mnist
# get training and testing vectors 
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,12))
plt.imshow(trainX[0], cmap=plt.get_cmap('Greys'), aspect='auto')
plt.colorbar()

In [ ]:
for i in range(9):
  plt.subplot(331+i)
  plt.imshow(trainX[i], cmap=plt.get_cmap('Greys'))

In [ ]:
# Check the shapes of the datasets
print('X_train: ' + str(trainX.shape))
print('Y_train: ' + str(trainy.shape))
print('X_test:  '  + str(testX.shape))
print('Y_test:  '  + str(testy.shape))

#### **Load in pytorch**

In [ ]:
import torchvision
import torchvision.transforms as transforms
import torch
import matplotlib.pyplot as plt
import numpy as np

## Select GPu if available
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu" 

#transforming the PIL Image to tensors
trainset = torchvision.datasets.FashionMNIST(root = "./data", train = True, download = True, transform = transforms.ToTensor())
testset = torchvision.datasets.FashionMNIST(root = "./data", train = False, download = True, transform = transforms.ToTensor())



In [ ]:
#loading the training data from trainset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle = False)
#loading the test data from testset
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

<font color='red'>**Task 1**: Print the first 6 images </font>

<font color='green'>**Hint**: the command *npimg = trainset.data[i].numpy()* converts the i-th data tensor in the training set to a numpy array. Then we can use the *imshow* as earlier to plot the images.</font>

In [ ]:
# Task 1 solution
for i in range(6):
  plt.subplot(331+i)
  npimg = trainset.data[i].numpy()
  plt.imshow(npimg, cmap=plt.get_cmap('Greys'))

In [ ]:
trainset.classes

In [ ]:
print(trainset.data.shape)
print(testset.data.shape)

<font color='green'>We continue to use pytorch for the remaining part</font>

#### **Let's see if the data is balanced**

In [ ]:
trainset.targets.unique(return_counts=True)

<font color='red'>**Task 2**: Do the same for the test set </font>

In [ ]:
# Task 2 solution
testset.targets.unique(return_counts=True)

### <font color='blue'>**Is it the same for MNIST?**</font>


In [ ]:
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [ ]:
type(mnist_trainset.data[0])


In [ ]:
print(mnist_trainset.data.shape)
print(mnist_testset.data.shape)



<font color='red'>**Task 3:** Plot first 6 images of MNIST</font>

In [ ]:
# Task 3 solution
for i in range(6):
  plt.subplot(331+i)
  npimg = mnist_trainset.data[i].numpy()
  plt.imshow(npimg, cmap=plt.get_cmap('Greys'))

In [ ]:
mnist_trainset.targets.unique(return_counts=True)

In [ ]:
mnist_testset.targets.unique(return_counts=True)

<font color='green'>**Answer**: MNIST data is nearly balanced. So we can assume that it is balanced.</font>

### <font color='blue'>**A rough comparison of the training and test set**</font>

In [ ]:
meansTrain = [torch.mean(mnist_trainset.data[mnist_trainset.targets == k,:,:].double()) for k in range(10)]
np.array(meansTrain)/255

In [ ]:
meansTest = [torch.mean(mnist_testset.data[mnist_testset.targets == k,:,:].double()) for k in range(10)]
np.array(meansTest)/255


In [ ]:
#import matplotlib.pyplot as plt
plt.plot(range(10), meansTrain, label="meansTrain")
plt.plot(range(10), meansTest, label="meansTest")
plt.legend()

<font color='red'>**Task 4**: Do the same for fashion MNIST</font>


In [ ]:
#Task 4 solution
meansTrain = [torch.mean(trainset.data[trainset.targets == k,:,:].double()) for k in range(10)]
np.array(meansTrain)/255
meansTest = [torch.mean(testset.data[testset.targets == k,:,:].double()) for k in range(10)]
np.array(meansTest)/255
plt.plot(range(10), meansTrain, label="meansTrain")
plt.plot(range(10), meansTest, label="meansTest")
plt.legend()


### <font color='blue'>**Some linear binary classification for MNIST**</font>

Start with digit '3' being 'positive' and digit '8' being negative

In [ ]:
positiveTrain = mnist_trainset.data[mnist_trainset.targets==3,:,:]
nPos = positiveTrain.shape[0]
negativeTrain = mnist_trainset.data[mnist_trainset.targets==8,:,:]
nNeg = negativeTrain.shape[0]
print(nPos, nNeg)

In [ ]:
positiveTrain[0].reshape(1, -1).shape # Converting a matrix to a row vector 

In [ ]:
positiveTrain[0].shape

In [ ]:
tempPos = torch.vstack([positiveTrain[i].reshape(-1) for i in range(nPos)])

In [ ]:
tempPos.shape

In [ ]:
import matplotlib.cm as cmap
plt.figure(figsize=(12,12))
plt.imshow(torch.transpose(tempPos, 0,1), cmap=cmap.hot, aspect='auto')
plt.colorbar()


In [ ]:
tempNeg = torch.vstack([negativeTrain[i].reshape(-1) for i in range(nNeg)])
tempNeg.shape

In [ ]:
a = torch.ones(nPos+nNeg) #This will be for the intercept (bias) term
a.reshape(-1,1)

In [ ]:
 A = torch.hstack((torch.ones(nPos+nNeg).reshape(-1,1), torch.vstack((tempPos, tempNeg))))

In [ ]:
A.shape

In [ ]:
plt.figure(figsize=(24,12))
plt.imshow(torch.transpose(A, 0,1).numpy(), cmap=cmap.hot, aspect='auto')
plt.colorbar()


In [ ]:
y = torch.hstack((torch.ones(nPos), -torch.ones(nNeg)))
print(y.shape)
y

We now minimize
$$
||y - A \beta ||^2
$$
with
$$
\hat{\beta} = A^\dagger y.
$$

In [ ]:
hat_beta = torch.mv(torch.pinverse(A), y)

In [ ]:
hat_beta.shape

In [ ]:
plt.plot(range(hat_beta.shape[0]), hat_beta,label = "β")
plt.xlabel("Index")
plt.ylabel("Value")
plt.legend()

In [ ]:
classify = lambda x : np.sign(torch.dot(hat_beta, torch.hstack((torch.tensor([1]), x.reshape(-1).float())))) # one line function definition in python

In [ ]:
(classify(positiveTrain[0]), classify(negativeTrain[0]))

In [ ]:
positiveTest = mnist_testset.data[mnist_testset.targets==3,:,:]
nPosTest = positiveTest.shape[0]
negativeTest = mnist_testset.data[mnist_testset.targets==8,:,:]
nNegTest = negativeTest.shape[0]
print(nPosTest, nNegTest)

In [ ]:
tempArr = np.array([classify(positiveTest[i]) for i in range(nPosTest)])
tempArr

In [ ]:
truePositives = np.count_nonzero(tempArr==1.0)
truePositives

In [ ]:
tempArr = np.array([classify(negativeTest[i]) for i in range(nNegTest)])


In [ ]:
trueNegatives = np.count_nonzero(tempArr==-1.0)
trueNegatives

<font color='red'>**Task 5:** What is the accuracy? What is the precision and recall? What is the $F_1$ score?</font>

In [ ]:
#Task 5 solution
accuracy = (truePositives + trueNegatives)/(nPosTest + nNegTest)
accuracy

Reminder:

$$
\text{Precision} = \frac{\big|\text{true positive}\big|}{\big|\text{true positive}\big| + \big|\text{false positive}\big|},
\qquad
\text{Recall} = \frac{\big|\text{true positive}\big|}{\big|\text{true positive}\big| + \big|\text{false negative}\big|}.
$$

In [ ]:
#Task 5 Solution
falsePositives = nNegTest - trueNegatives
falseNegatives = nPosTest - truePositives
print(falsePositives, falseNegatives)

In [ ]:
#Task 5 Solution
precision = truePositives/(truePositives+falsePositives)
precision

In [ ]:
#Task 5 Solution
recall = truePositives/(truePositives+falseNegatives)
recall

In [ ]:
#Task 5 Solution
F1 = 1/np.mean([1/precision,1/recall]) #This is the harmonic mean
F1


<font color='red'>**Task 6:** Repeat the above to make a classifier that distingiushes between the `0` digit and `3` digit</font>


In [ ]:
# Task5 solution
# Do it as a homework.
# You will observe that accuracy, precision, recall, and F1 score will be very high

### <font color='blue'>**CIFAR10**</font>
See [CIFAR10 website](https://www.cs.toronto.edu/~kriz/cifar.html)

In [ ]:
cifar10_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=None)
cifar10_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=None)

In [ ]:
cifar10_testset.classes

In [ ]:
cifar10_trainset.targets = torch.tensor(cifar10_trainset.targets).clone().detach()
cifar10_testset.targets = torch.tensor(cifar10_testset.targets).clone().detach()


In [ ]:
cifar10_trainset.targets.unique(return_counts=True)

In [ ]:
cifar10_testset.targets.unique(return_counts=True)

In [ ]:
cifar10_trainset.data.shape

In [ ]:
# First image
cifar10_trainset.data[0]/255

In [ ]:
plt.figure(figsize=(15,12))
plt.imshow(cifar10_trainset.data[0], cmap=cmap.hot, aspect='auto')
plt.colorbar()

In [ ]:
cifar10_trainset.targets[0].item()

In [ ]:
cifar10_trainset.classes[cifar10_trainset.targets[0].item()]

In [ ]:
plt.figure(figsize=(6,6))
for i in range(6):
  plt.subplot(331+i)
  plt.imshow(cifar10_trainset.data[i])

In [ ]:
plt.imshow(cifar10_trainset.data[0,:,:, 0], aspect='auto', cmap=plt.get_cmap('Reds'))
plt.colorbar()

In [ ]:
plt.imshow(cifar10_trainset.data[0,:,:, 1], aspect='auto', cmap=plt.get_cmap('Greens'))
plt.colorbar()

In [ ]:
plt.imshow(cifar10_trainset.data[0,:,:, 2], aspect='auto', cmap=plt.get_cmap('Blues'))
plt.colorbar()

In [ ]:
from matplotlib.animation import FuncAnimation
from matplotlib import rc
fig, ax = plt.subplots(figsize=(3, 3))

frogs = cifar10_trainset.data[cifar10_trainset.targets == 6, :, :, :][0:100]

def update(i):
    im_normed = frogs[i]
    ax.imshow(im_normed)
    ax.set_title("Frog :"+str(i+1), fontsize=20)
    ax.set_axis_off()


anim = FuncAnimation(fig, update, frames=np.arange(0, frogs.shape[0]), interval=200)
rc('animation', html='jshtml')
anim    # or HTML(anim.to_jshtml())

<font color='red'>**Task 7:** Create an animation that goes through the first 100 images and presents the label of each image.</font>

In [ ]:
#Task 7 Solution
fig, ax = plt.subplots(figsize=(3, 3))


def update(i):
    im_normed = cifar10_trainset.data[i]
    ax.imshow(im_normed)
    ax.set_title(str(i+1)+' :'+cifar10_trainset.classes[cifar10_trainset.targets[i].item()], fontsize=20)
    ax.set_axis_off()


anim = FuncAnimation(fig, update, frames=np.arange(0, 100), interval=300)
rc('animation', html='jshtml')
anim    # or HTML(anim.to_jshtml())